In [1]:
# Imports
import pandas as pd
import numpy as np
import pickle

In [2]:
# Read Diagnostic Data
data = pd.read_csv('ADNIMERGE.csv')

/Users/jnachbar/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (18,19,20,103,104) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
# We only care about ID, Exam Data, Baseline, and Current
dx_data = data[['PTID', 'EXAMDATE', 'DX_bl', 'DX']]

In [4]:
# We only want Diagnoses which have changed
m = dx_data.query('DX != DX_bl')

In [5]:
# Convert to numpy so I can actually work with it
arr = m.to_numpy()

In [9]:
# If we changed from MCI to AD
last = ''
count = 0
arr_s = []
for val in arr:
    if val[0] == last:
        count += 1
    else:
        count = 0
        last = val[0]
    if val[2] == 'LMCI' and val[3] == 'Dementia':
        # if this is the case, scan backwards to figure out how many images we have before and add that to val
        tmp = list(val)
        tmp.append(count)
        arr_s.append(tmp)

In [12]:
# only capture the first instance, so the breakthrough Diagnosis
arr_final_AD = []
last_names = set()
for val in arr_s:
    if val[0] not in last_names:
        arr_final_AD.append(list(val))
        last_names.add(val[0])   

In [13]:
# measure how many previous visits each patient has
count_arr = np.zeros(10)
for val in arr_final_AD:
    count_arr[val[4]] += 1
    
print(count_arr)

[105.  44.  68.  34.  27.  13.   5.   2.   3.   0.]


In [14]:
# no need to include redundant data
for i in range(len(arr_final_AD)):
    arr_final_AD[i] = [arr_final_AD[i][0], arr_final_AD[i][1], arr_final_AD[i][4]]

In [15]:
# Save it as a csv
pd.DataFrame(arr_final_AD).to_csv("./MCI_to_AD.csv")

In [ ]:
# capture all patients who started as MCI and remained as MCI
# take all patients who started as MCI, and subtract arr_f from it
MCI_arr = dx_data.to_numpy()
arr_m = []
for val in MCI_arr:
    if val[2] == 'LMCI':
        arr_m.append(list(val))
        

In [20]:
# go through each value
# if we get dementia or nan, skip everything on the current ID, and do not append
# otherwise, when we change, append

# x is whatever nan is in the original array. Super hacky, but whatever
x = arr_m[4991][3]

arr_final_MCI = []
last_names = set()
last_val = ''
count = 0
append = True
# go through each entry. When the subject ID changes (or is at an end), if there were no instances of dementia, 
# append that patient ID and the count
for val in arr_m:
    if val[0] not in last_names:
        if append:
            tmp = list(last_val)
            tmp.append(count)
            arr_final_MCI.append(tmp)
        last_names.add(val[0])
        count = 0
        append = True
    if val[3] in ['Dementia', x]:
        append = False
    last_val = val
    count += 1
    

In [19]:
arr_m

[['022_S_0004', '2005-11-08', 'LMCI', 'MCI'],
 ['022_S_0004', '2006-05-02', 'LMCI', 'MCI'],
 ['022_S_0004', '2006-11-14', 'LMCI', 'MCI'],
 ['022_S_0004', '2007-05-14', 'LMCI', 'MCI'],
 ['022_S_0004', '2008-11-18', 'LMCI', 'MCI'],
 ['100_S_0006', '2005-11-29', 'LMCI', 'MCI'],
 ['100_S_0006', '2006-06-01', 'LMCI', 'MCI'],
 ['100_S_0006', '2006-11-20', 'LMCI', 'MCI'],
 ['100_S_0006', '2007-05-15', 'LMCI', 'MCI'],
 ['100_S_0006', '2008-12-08', 'LMCI', 'MCI'],
 ['035_S_0033', '2006-12-14', 'LMCI', 'MCI'],
 ['035_S_0033', '2009-01-09', 'LMCI', 'MCI'],
 ['067_S_0038', '2005-12-15', 'LMCI', 'MCI'],
 ['067_S_0038', '2006-06-23', 'LMCI', 'MCI'],
 ['067_S_0038', '2006-12-07', 'LMCI', 'MCI'],
 ['007_S_0041', '2006-05-15', 'LMCI', 'MCI'],
 ['023_S_0042', '2005-11-10', 'LMCI', 'MCI'],
 ['023_S_0042', '2006-05-02', 'LMCI', 'MCI'],
 ['023_S_0042', '2006-11-09', 'LMCI', 'Dementia'],
 ['023_S_0042', '2007-05-10', 'LMCI', 'Dementia'],
 ['023_S_0042', '2007-11-30', 'LMCI', 'Dementia'],
 ['023_S_0042', '20

In [23]:
arr_final_MCI

[[0],
 ['022_S_0004', '2008-11-18', 'LMCI', 'MCI', 5],
 ['100_S_0006', '2008-12-08', 'LMCI', 'MCI', 5],
 ['035_S_0033', '2009-01-09', 'LMCI', 'MCI', 2],
 ['067_S_0038', '2006-12-07', 'LMCI', 'MCI', 3],
 ['007_S_0041', '2006-05-15', 'LMCI', 'MCI', 1],
 ['022_S_0044', '2006-05-19', 'LMCI', 'MCI', 1],
 ['067_S_0045', '2006-07-13', 'LMCI', 'MCI', 2],
 ['099_S_0051', '2008-12-11', 'LMCI', 'MCI', 6],
 ['099_S_0060', '2007-07-12', 'LMCI', 'MCI', 4],
 ['018_S_0080', '2007-01-09', 'LMCI', 'MCI', 3],
 ['018_S_0087', '2007-08-01', 'LMCI', 'MCI', 4],
 ['130_S_0102', '2007-07-13', 'LMCI', 'MCI', 2],
 ['018_S_0103', '2006-08-08', 'LMCI', 'MCI', 2],
 ['136_S_0107', '2010-06-29', 'LMCI', 'MCI', 6],
 ['127_S_0112', '2010-01-28', 'LMCI', 'CN', 7],
 ['027_S_0116', '2010-02-17', 'LMCI', 'MCI', 5],
 ['128_S_0138', '2007-02-21', 'LMCI', 'MCI', 3],
 ['018_S_0142', '2010-02-26', 'LMCI', 'MCI', 7],
 ['037_S_0150', '2009-03-25', 'LMCI', 'MCI', 6],
 ['018_S_0155', '2008-03-06', 'LMCI', 'MCI', 4],
 ['098_S_0160',

In [25]:
# measure how many previous visits each patient has
count_arr = np.zeros(11)
for val in arr_final_MCI:
    count_arr[val[4]] += 1
    
print(count_arr)

IndexError: list index out of range

In [19]:
# no need to include redundant data
for i in range(len(arr_final_MCI)):
    arr_final_MCI[i] = [arr_final_MCI[i][0], arr_final_MCI[i][1], arr_final_MCI[i][4]]

In [20]:
pd.DataFrame(arr_final_MCI).to_csv("./MCI_to_MCI.csv")